# Carga de datos

In [3]:
import pandas as pd
# Carga
df = pd.read_csv('data/departamentos_clean.csv')
df.shape

(2349, 13)

In [ ]:
df.head()

## Modelo SVM

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn import svm

In [ ]:
# Para usar los métodos SVM las variables categoricas "comuna" y "estacion_cercana" se transforman a variables dummys
df = pd.get_dummies(df)

In [ ]:
# Separación de etiqueta Y y caracteristicas X
y = df.iloc[:,0]
X = df.iloc[:,1:]
# Subconjunto de entrenamiento y testing de 80 y 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Prueba automatica de diferentes modelos SVM

In [ ]:
for kernel in ['linear', 'poly', 'rbf', 'sigmoid'] :
    print('====> Kernel : ', kernel)
    for C in [0.01, 0.1, 1, 1.5, 5, 10, 20] :
        print('==> C : ', C)
        clf_iter = svm.SVC(kernel=kernel, C=C)
        accuracy = cross_val_score(clf_iter, X, y, cv=5)
        print("Precisión promedio: ", accuracy.mean() * 100)
        clf_iter.fit(X_train, y_train)
        y_test_pred = clf_iter.predict(X_test)
        f1 = f1_score(y_test, y_test_pred, average='macro')
        confusion_matrix(y_test, y_test_pred)
        print('f1 : ', f1)
        aucscore = roc_auc_score(y_test, y_test_pred, average='macro')
        print('auc : ', aucscore)

# Testing

### Probando la Estandarización

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

print(scaler.fit(df[['precio', 'distancia_estacion']]))

print(scaler.mean_)

print(scaler.transform(df[['precio', 'distancia_estacion']]))

#print(scaler.transform([[2, 2]]))

StandardScaler()
[1.28762794e+06 1.27533206e+03]
[[-4.92310153e-01 -7.72657037e-01]
 [ 1.18303257e-01 -7.61022416e-01]
 [-9.66061937e-01  1.41108216e-01]
 ...
 [-6.18643962e-01  1.54352756e+00]
 [-5.97912802e-04  1.54352756e+00]
 [-3.39346230e-01 -6.36621466e-01]]


In [10]:
df[['precio', 'dormitorios']].head()

,precio,dormitorios
0,820000.0,3.0
1,1400000.0,4.0
2,370000.0,2.0
3,1898413.0,3.0
4,1500000.0,4.0


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

print(scaler.fit(df[['precio', 'dormitorios']]))

print(scaler.mean_)

print(scaler.transform(df[['precio', 'dormitorios']]))

StandardScaler()
[1.28762794e+06 2.50191571e+00]
[[-4.92310153e-01  4.78122161e-01]
 [ 1.18303257e-01  1.43804435e+00]
 [-9.66061937e-01 -4.81800024e-01]
 ...
 [-6.18643962e-01 -1.44172221e+00]
 [-5.97912802e-04  1.43804435e+00]
 [-3.39346230e-01  4.78122161e-01]]


### Usando Pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline